# quakejs web services



## master server



### module index

quakejs web master?


#### the code





In [ ]:
var master = root.master = {};

module.exports = master




### connection

quakejs connection?


#### the code



In [ ]:
var importer = require('../Core')
var WebSocket = require('ws')
var {_formatOOB, _stripOOB} = importer.import('quakejs utilities')
var {_parseServers, _parseInfoResponse} = importer.import('quakejs parsing')

function connect(address, port, callback) {
	var self = this;

	var errored = false;
	var ws = new WebSocket('ws://' + address + ':' + port);
	ws.binaryType = 'arraybuffer';

	ws.onopen = function () {
		var buffer = _formatOOB('subscribe');

		ws.send(buffer);
	};

	ws.onmessage = function (event) {
		var data = _stripOOB(event.data);

		if (data.indexOf('getserversResponse') === 0) {
			data = data.substr(18);

			var servers = _parseServers(data);

			callback(null, servers);
		}
	};

	ws.onclose = function () {
		if (!errored) {
			callback(new Error('Connection to master server lost.'));
			errored = true;
		}
	};
};

function scanServer(server, callback) {
	var self = this;

	var ws = new WebSocket('ws://' + server.addr + ':' + server.port);
	ws.binaryType = 'arraybuffer';
	var start, end;
	var finish = function (err, info) {
		if (callback) {
			callback(err, info);
			callback = null;
		}
		ws.close();
	};

	ws.onopen = function () {
		start = window.performance.now();

		var buffer = _formatOOB('getinfo');

		ws.send(buffer);
	};

	ws.onmessage = function (event) {
		end = window.performance.now();

		var data = _stripOOB(event.data);
		var info;
		try {
			info = _parseInfoResponse(data);
		} catch (err) {
			finish(err);
			return;
		}
		info.ping = parseInt(end - start, 10);
		finish(null, info);
	};

	ws.onclose = function (ev) {
		finish(new Error(ev.reason));
	};
};

module.exports = {
    connect,
    scanServer
}


### parsing

quakejs parsing?


#### the code



In [ ]:

function _parseInfoString(str) {
	var data = {};

	if (str) {
		// trim the incoming string
		str = str.replace(/^\\+|\\+$/g, '');

		var split = str.split('\\');

		for (var i = 0; i < split.length - 1; i += 2) {
			var key = split[i];
			var value = split[i+1];
			data[key] = value;
		}
	}

	return data;
};

function _parseServers(str) {
	var servers = [];

	str = str.replace(/\\EOT$|^\\/g, '');

	if (str) {
		var split = str.split('\\');

		for (var i = 0; i < split.length; i++) {
			var info = split[i];
			var addr = (info.charCodeAt(0) & 0xff).toString() + '.' + (info.charCodeAt(1) & 0xff).toString() + '.' +
				(info.charCodeAt(2) & 0xff).toString() + '.' + (info.charCodeAt(3) & 0xff).toString();
			var port = ((info.charCodeAt(4) & 0xff) << 8) | (info.charCodeAt(5) & 0xff);

			servers.push({ addr: addr, port: port });
		}
	}

	return servers;
};

function _parsePlayers(str) {
	var players = [];

	if (str) {
		// trim the incoming string
		str = str.replace(/^\n+|\n+$/g, '');

		var split = str.split('\n');

		for (var i = 0; i < split.length; i++) {
			var pinfo = split[i];
			var psplit = pinfo.match(/\S+|"[^"]+"/g);  // split on space, combining quoted items

			players.push({
				frags: parseInt(psplit[0], 10),
				ping: parseInt(psplit[1], 10),
				name: psplit[2]
			});
		}
	}

	return players;
};

function _parseStatusResponse(data) {
	var self = this;
	if (data.indexOf('statusResponse\n') !== 0) {
		throw new Error('Invalid getstatus response: ' + data);
	}
	data = data.substr(15);

	var idx = data.indexOf('\n');
	var variableData = idx !== -1 ? data.substr(0, idx) : data;
	var playerData = idx !== -1 ? data.substr(idx) : null;

	var info = _parseInfoString(variableData);
	info.players = _parsePlayers(playerData);
	return info;
};

function _parseInfoResponse(data) {
	var self = this;
	if (data.indexOf('infoResponse\n') !== 0) {
		throw new Error('Invalid getinfo response: ' + data);
	}
	data = data.substr(13);
	var info = _parseInfoString(data);

	// Compute the number of bots for the template
	info.g_botplayers = info.clients - info.g_humanplayers;

	return info;
};

module.exports = {
    _parseServers,
    _parsePlayers,
    _parseStatusResponse,
    _parseInfoResponse
}


### utilities

quakejs utilities?


#### the code




In [ ]:

function _formatOOB(data) {
	var str = '\xff\xff\xff\xff' + data + '\x00';

	var buffer = new ArrayBuffer(str.length);
	var view = new Uint8Array(buffer);

	for (var i = 0; i < str.length; i++) {
		view[i] = str.charCodeAt(i);
	}

	return buffer;
};

function _stripOOB(buffer) {
	var view = new DataView(buffer);

	if (view.getInt32(0) !== -1) {
		return null;
	}

	var str = '';
	var i_start = 4; /* ignore leading -1 */
	var i_end = buffer.byteLength;

	/* ignore trailing whitespace */
	while (i_end > i_start && view.getUint8(i_end - 1) <= ' '.charCodeAt(0)) {
		--i_end;
	}

	for (var i = i_start; i < i_end; i++) {
		var c = String.fromCharCode(view.getUint8(i));
		str += c;
	}

	return str;
};

module.exports = {
    _formatOOB,
    _stripOOB
}


### testing

test quakejs master?


#### the code




In [ ]:
var importer = require('../Core')
var master = importer.import('quakejs connection')

var host = 'localhost'
var port = 27950

function testServers() {
    master.connect(host, port, function (err, servers) {
        if (err) {
            self.error(err);

            // attempt to reconnect in a minute
            setTimeout(testServers, 60000);

            return;
        }

        servers.forEach(function (server) {
            master.scanServer(server, function (err, info) {
                if (err) {
                    console.log('Failed to scan ' + server.addr + ':' + server.port + ', ' + err.message);
                    console.log(server);
                    return;
                }

                console.log(server, info);
            });
        });
    });
}

module.exports = testServers
